# Basic Spark SQL Usage

### Example of using Spark SQL with Stroom DataFrame

#### Prerequisites
This notebook is designed to work with a Stroom server process running on `localhost`, into which the example data has been loaded (e.g. by running the gradle task `setupSampleData`).

You must set the environmental variable `STROOM_API_KEY` to the API token associated with a suitably privileged Stroom user account before starting the Jupyter notebook server process.

#### Setup
Import standard utility classes/functions, including JSON handling XSLT.

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col
from IPython.display import display
from pyspark.sql import SparkSession

#### Create a schema using XPaths

N.B. XPath @* is used to extract both StreamId and EventId from the Event, and placed into a single field.
This field has unique values, handy for working with SQL.

In [2]:
mySchema = StructType([StructField("user", StringType(), True, 
                                   metadata={"get": "EventSource/User/Id"}), 
                       StructField("operation", StringType(), True, 
                                   metadata={"get": "EventDetail/TypeId"}),
                     StructField("eventid", StringType(), False,
                                metadata={"get": "@*"})])

In [3]:
stroomDf = spark.read.format('stroom.spark.datasource.StroomDataSource').load(
        token=os.environ['STROOM_API_KEY'],host='localhost',protocol='http',
        uri='api/stroom-index/v2',
        index='57a35b9a-083c-4a93-a813-fc3ddfe1ff44',pipeline='bb25824e-6369-464a-81e1-876ffe3b95a0',
        schema=mySchema).select('eventid','user','operation','idxUserId')

In [4]:
display(stroomDf.limit(5).toPandas().head())

,eventid,user,operation,idxUserId
0,742|1,user1,0001,
1,740|3,user3,0001,
2,777|1,user1,0001,
3,777|2,user2,0001,
4,777|3,user3,0001,


#### Using Spark SQL

In order to start actually writing SQL queries, it is necessary to create a temporary view onto the 
Stroom DataFrame created above.

Results are returned as DataFrames themselves, making further operations possible.

In [5]:
stroomDf.createOrReplaceTempView("userops")
sqlDf = spark.sql("select * from userops where user='user1' and operation='0001'")

In [6]:
display(sqlDf.limit(5).toPandas().head())

,eventid,user,operation,idxUserId
0,742|1,user1,0001,
1,777|1,user1,0001,
2,778|11,user1,0001,
3,779|1,user1,0001,
4,779|11,user1,0001,


In [7]:
sqlDf2 = spark.sql("select user,operation, count (eventid) as events from userops \
                    where idxUserId != 'User1' group by user, operation \
                    order by events desc")
display(sqlDf2.toPandas())

,user,operation,events
0,user10,0001,1421
1,user6,0001,1402
2,user9,0001,1380
3,user7,0001,1374
4,user8,0001,1358
5,user3,0001,1355
6,user4,0001,1342
7,user5,0001,1334
8,user2,0001,1328
9,user10,Logon,10
